In [64]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [65]:
# Load the dataset
df = pd.read_csv("/kaggle/input/restaurant-dataset/Restaurant_data.csv")

# Display first few rows
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [66]:
# Shape and info
print("Shape:", df.shape)
df.info()

Shape: (9551, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Swi

In [67]:
# Check for missing values
print("\nMissing Values:\n", df.isnull().sum())


Missing Values:
 Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64


In [68]:
# Statistical summary
df.describe()

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Price range,Aggregate rating,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,1199.210763,1.804837,2.666370,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,16121.183073,0.905609,1.516378,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,1.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,250.000000,1.000000,2.500000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,400.000000,2.000000,3.200000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,700.000000,2.000000,3.700000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,800000.000000,4.000000,4.900000,10934.000000


In [69]:
# Preprocessing
df.drop_duplicates(inplace=True)

# Ensure no NaN and everything is string type before combining
for col in ['Cuisines', 'City', 'Locality', 'Rating text']:
    df[col] = df[col].fillna('').astype(str)

In [70]:
# Preprocess dataframe: lowercasing City, Locality, and Cuisines for better matching
df['City_lower'] = df['City'].str.lower()
df['Locality_lower'] = df['Locality'].str.lower()
df['Cuisines_lower'] = df['Cuisines'].str.lower()

In [93]:
# recommendation function
def recommend_restaurants(user_cuisine, user_city, user_locality, user_price_range):
    # Filter based on user preferences
    recommendations = df.copy()
    
    # Filter by cuisine
    if user_cuisine.lower() != 'any':
        recommendations = recommendations[recommendations['Cuisines'].str.contains(user_cuisine, case=False, na=False)]
    
    # Filter by city
    if user_city.lower() != 'any':
        recommendations = recommendations[recommendations['City'].str.contains(user_city, case=False, na=False)]
    
    # Filter by locality (optional)
    if user_locality.lower() != 'any':
        recommendations = recommendations[recommendations['Locality'].str.contains(user_locality, case=False, na=False)]
    
    # Filter by price range
    if user_price_range.lower() != 'any':
        try:
            price_range = int(user_price_range)
            recommendations = recommendations[recommendations['Price range'] == price_range]
        except ValueError:
            pass  # If price range is invalid, skip filtering

    # Handle case where no recommendations are found
    if recommendations.empty:
        return "No restaurants found matching your preferences. Please try again with different criteria."

    # Add currency to price column
    if 'Currency' in recommendations.columns:
        recommendations['Price for 2'] = recommendations['Currency'] + ' ' + recommendations['Average Cost for two'].astype(str)
    else:
        recommendations['Price for 2'] = recommendations['Average Cost for two'].astype(str)

    # Columns to display
    required_columns = ['Restaurant Name', 'Locality', 'City', 'Cuisines', 'Price for 2', 'Aggregate rating']
    recommendations = recommendations[required_columns]

    # ✅ Strictly format Aggregate rating to 1 decimal string
    recommendations['Aggregate rating'] = recommendations['Aggregate rating'].apply(lambda x: f"{x:.1f}")

    return recommendations.head(5)

In [94]:
# User inputs
user_cuisine = input("Enter preferred cuisine (or type 'Any'): ")
user_city = input("Enter your city: ")
user_locality = input("Enter your locality (or type 'Any'): ")
user_price_range = input("Enter preferred price range (1 to 4) or type 'Any': ")

# Get recommendations
recommendations = recommend_restaurants(user_cuisine, user_city, user_locality, user_price_range)

# Display recommendations
print("\n Near-by Recommended Restaurants for You:")

if isinstance(recommendations, pd.DataFrame):
    display(recommendations.style.set_properties(**{
        'background-color': '#f9f9f9',
        'color': 'black',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'text-align': 'center'
    }).set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#4CAF50'), ('color', 'white'), ('text-align', 'center')]
    }]).hide(axis="index"))
else:
    print(recommendations)

Enter preferred cuisine (or type 'Any'):  french
Enter your city:  makati
Enter your locality (or type 'Any'):  any
Enter preferred price range (1 to 4) or type 'Any':  any



 Near-by Recommended Restaurants for You:


Restaurant Name,Locality,City,Cuisines,Price for 2,Aggregate rating
Le Petit Souffle,"Century City Mall, Poblacion, Makati City",Makati City,"French, Japanese, Desserts",Botswana Pula(P) 1100,4.8
